In [ ]:
!pip3 install transformers datasets

In [ ]:
from transformers import BertForSequenceClassification,AutoTokenizer,Trainer,TrainingArguments
from datasets import load_dataset
import os

In [ ]:
 os.environ["WANDB_DISABLED"] = "true"

 # Load dataset
 dataset = load_dataset("imdb")

 # Load model and tokenizer
 model = BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)
 tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

 # Preprocess data
 def preprocess_function(examples):
  return tokenizer(examples['text'],truncation=True,padding=True)

 encoded_dataset = dataset.map(preprocess_function,batched=True, batch_size=64)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
 # Training arguments
 training_args = TrainingArguments(
  output_dir="./results",
  evaluation_strategy = "epoch",
  save_strategy = "epoch",
  learning_rate = 2e-5,
  per_device_train_batch_size = 16,
  num_train_epochs = 1
 )

 trainer = Trainer(
  model = model,
  args = training_args,
  train_dataset = encoded_dataset["train"],
  eval_dataset = encoded_dataset["test"]
 )

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer.train()

In [ ]:
# 保存微调后的模型
trainer.save_model("./fine_tuned_bert_imdb")